In [1]:
import pandas as pd
# Set max columns to display
pd.set_option('display.max_columns', None)

import numpy as np
from sklearn import preprocessing as pre
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



In [2]:

# Importing CSV files
df_CDunit = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\1 Preprocess\Continuous Data\cont_unitData_clean.csv")
df_AlCon = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\1 Preprocess\Continuous Data\cont_425Data_clean.csv")
df_FB554 = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\1 Preprocess\Continuous Data\cont_554Data_clean.csv")


In [3]:
print(df_CDunit.describe())
print(df_AlCon.describe())
print(df_FB554.describe())


           AYC55580       DI55102       DI55152       DI55580       FC42428  \
count  49239.000000  49239.000000  49239.000000  49239.000000  49239.000000   
mean      11.266268      0.929918      0.941470      0.997989  35926.808144   
std        4.622232      0.067214      0.039545      0.049746   5103.120658   
min       -2.825930      0.794541      0.818505      0.844666  16906.900000   
25%        9.714050      0.892178      0.915156      0.966571  33984.800000   
50%       10.767700      0.940556      0.937905      0.997402  37734.700000   
75%       13.391700      0.980936      0.961704      1.032280  39364.100000   
max       25.133700      1.068230      1.063430      1.135600  49959.600000   

            FC52018       FC55003       FC55009       FC55102       FC55152  \
count  49239.000000  49239.000000  49239.000000  49239.000000  49239.000000   
mean   32706.413168   5961.217965    869.257900  44691.095554  40539.984571   
std     6023.684555    832.099515    608.120131   6

In [4]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: object
Data type for 'Date' column in df_FB554: object
Data type for 'Date' column in df_AlCon: object


In [5]:
df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')
df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')


In [6]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [7]:
print(df_CDunit.columns)
print(df_FB554.columns)
print(df_AlCon.columns)


Index(['Date', 'AYC55580', 'DI55102', 'DI55152', 'DI55580', 'FC42428',
       'FC52018', 'FC55003', 'FC55009', 'FC55102', 'FC55152', 'FC55552',
       'FC55555', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'FYC55553',
       'II52554', 'LC52572', 'LC55553', 'LC55555', 'LC55557', 'LC55568',
       'LC90366', 'LC90368', 'PI55004', 'PI55020', 'PI55560', 'TC52015',
       'TC55552', 'TC55553', 'TC55555', 'TC55566', 'TI40050', 'TI52014',
       'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021',
       'TI55023', 'VI52558B'],
      dtype='object')
Index(['Date', 'Butanol', 'Decanol', 'Ethanol', 'Hexanol', 'Octanol'], dtype='object')
Index(['Date', '425 %Al'], dtype='object')


In [8]:
def apply_rolling_average_to_df(df, rolling_size):
    # Ensure 'Date' is the index if it's not already
    if df.index.name != 'Date':
        df = df.set_index('Date')

    # Apply rolling average to all columns
    rolled_df = df.rolling(window=rolling_size, min_periods=1).mean()

    # Reset index to make 'Date' a column again
    rolled_df = rolled_df.reset_index()

    return rolled_df



In [9]:
def apply_time_shift_by_hours(df, shift_hours):
    """
    Shifts the DataFrame's datetime index by the specified number of hours.

    :param df: DataFrame with 'Date' as its datetime index or column.
    :param shift_hours: Number of hours to shift. Can be positive (forward) or negative (backward).
    :return: Shifted DataFrame.
    """
    # Convert 'Date' to datetime and set as index if it's not already
    if df.index.name != 'Date':
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.set_index('Date')

    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)

    # Shift the DataFrame's index by the specified number of hours
    df.index = df.index + pd.Timedelta(hours=shift_hours)

    return df

In [10]:
# # Usage Examples
# shift_hours_AlCon = 1  # Negative shift for df_AlCon (e.g., -5 hours backward)
# shift_hours_FB554 = 5   # Positive shift for df_FB554 (e.g., 5 hours forward)

# shifted_df_AlCon = apply_time_shift_by_hours(df_AlCon, shift_hours_AlCon)
# print("Shifted df_AlCon:")
# print(shifted_df_AlCon.head())

# shifted_df_FB554 = apply_time_shift_by_hours(df_FB554, shift_hours_FB554)
# print("\nShifted df_FB554:")
# print(shifted_df_FB554.head())

In [10]:
def join_df_FB554_to_df_CDunit(df_CDunit, df_FB554):
    # Ensure 'Date' columns are datetime objects and sort DataFrames
    df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
    df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')

    df_CDunit = df_CDunit.dropna(subset=['Date']).sort_values('Date')
    df_FB554 = df_FB554.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df = pd.merge_asof(df_FB554, df_CDunit, on='Date', direction='nearest')

    return combined_df

def join_df_AlCon_to_combined_df(combined_df, df_AlCon):
    # Ensure 'Date' columns are datetime objects and sort DataFrames
    combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
    df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')

    combined_df = combined_df.dropna(subset=['Date']).sort_values('Date')
    df_AlCon = df_AlCon.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df_all = pd.merge_asof(df_AlCon, combined_df, on='Date', direction='nearest')
    
    return combined_df_all


In [11]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)



Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [14]:
def apply_negative_shift_hours(shift_hours):
    return [-hour for hour in shift_hours]

# Rolling sizes ranges
rolling_size_CDunit = range(4, 11, 2)  # Even rolling sizes from 4 to 10 
rolling_size_FB554 = range(4, 11, 2)  # Even rolling sizes from 4 to 10 
rolling_size_AlCon = range(2, 31, 2)  # Even rolling sizes from 2 to 30 

# Shift hours ranges
shift_hours_AlCon = apply_negative_shift_hours(range(2, 9, 2))  # Negative shifts from -2 to -8
shift_hours_FB554 = range(2, 9, 2)   # Positive shifts from 2 to 8

# Precompute rolling averages for each DataFrame and each rolling size
precomputed_rolls = {
    "CDunit": {size: apply_rolling_average_to_df(df_CDunit, size) for size in rolling_size_CDunit},
    "FB554": {size: apply_rolling_average_to_df(df_FB554, size) for size in rolling_size_FB554},
    "AlCon": {size: apply_rolling_average_to_df(df_AlCon, size) for size in rolling_size_AlCon}
}

## Modified process_data function
def process_data():
    iteration_count = 0
    results = pd.DataFrame(columns=['Iteration', 'Rolling Sizes CDunit', 'Rolling Sizes FB554', 'Rolling Sizes AlCon',
                                    'Shift Hours AlCon', 'Shift Hours FB554', 'R-squared', 'Adj R-Squared', 
                                    'F-statistic', 'Prob (F-statistic)'])

    for size_CDunit in rolling_size_CDunit:
        for size_FB554 in rolling_size_FB554:
            for size_AlCon in rolling_size_AlCon:
                for shift_hour_AlCon in shift_hours_AlCon:
                    for shift_hour_FB554 in shift_hours_FB554:
                        iteration_count += 1

                        # Retrieve rolled dataframes
                        rolled_df_CDunit = precomputed_rolls["CDunit"][size_CDunit]
                        rolled_df_FB554 = precomputed_rolls["FB554"][size_FB554]
                        rolled_df_AlCon = precomputed_rolls["AlCon"][size_AlCon]

                        # Combine df_CDunit and df_FB554 to create combined_df
                        combined_df = join_df_FB554_to_df_CDunit(rolled_df_CDunit, rolled_df_FB554)

                        # Combine combined_df with rolled_df_AlCon to create combined_df_all
                        combined_df_all = join_df_AlCon_to_combined_df(combined_df, rolled_df_AlCon)

                        # Drop 'Date' column before modeling
                        combined_df_all = combined_df_all.drop(columns=['Date'], errors='ignore')

                        # Splitting into train and test
                        X = combined_df_all.drop('Butanol', axis=1)
                        y = combined_df_all['Butanol']
                        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                        # Train model
                        model = sm.OLS(y_train, X_train).fit()

                        # Store the results instead of printing
                        iteration_results = {
                            'Iteration': iteration_count,
                            'Rolling Sizes CDunit': size_CDunit,
                            'Rolling Sizes FB554': size_FB554,
                            'Rolling Sizes AlCon': size_AlCon,
                            'Shift Hours AlCon': shift_hour_AlCon,
                            'Shift Hours FB554': shift_hour_FB554,
                            'R-squared': model.rsquared,
                            'Adj R-Squared': model.rsquared_adj,
                            'F-statistic': model.fvalue,
                            'Prob (F-statistic)': model.f_pvalue
                        }
                        results = results.append(iteration_results, ignore_index=True)

                        # Print only the iteration count
                        print(f"Iteration: {iteration_count}")

    return results

# Call the function to process and evaluate the data
model_results = process_data()

C:\Users\saust\AppData\Local\Temp\ipykernel_86628\710981374.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(iteration_results, ignore_index=True)


Iteration: 1


C:\Users\saust\AppData\Local\Temp\ipykernel_86628\710981374.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(iteration_results, ignore_index=True)


Iteration: 2


C:\Users\saust\AppData\Local\Temp\ipykernel_86628\710981374.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(iteration_results, ignore_index=True)


...

C:\Users\saust\AppData\Local\Temp\ipykernel_86628\710981374.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(iteration_results, ignore_index=True)


Iteration: 3838


C:\Users\saust\AppData\Local\Temp\ipykernel_86628\710981374.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(iteration_results, ignore_index=True)


Iteration: 3839
Iteration: 3840


C:\Users\saust\AppData\Local\Temp\ipykernel_86628\710981374.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(iteration_results, ignore_index=True)


In [15]:
# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
model_results.to_csv('model_results_cd.csv', index=False)